In [ ]:
import cv2
import numpy as np
from picamera2 import Picamera2
from matplotlib import pyplot as plt

In [ ]:
picam2 = Picamera2()
config = picam2.create_still_configuration(main={"size": (1640, 1232)})
picam2. configure(config)
picam2.start()

In [ ]:
try:
    # 이미지 한장 캡처
    frame = picam2.capture_array()

    # OpenCV 형식으로 변환
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    plt.figure(figsize=(10, 8))
    plt.imshow(frame)
    plt.title("Captured Image: {frame.shape[1]}x{frame.shape[0]}")
    plt.axis('off')
    plt.show()

    print("이미지 캡처 완료")

finally:
    picam2.stop()

# 이미지를 JPEG로 압축해 TARGET_IP 로 전송

In [ ]:
import cv2
import socket
import numpy as np
from picamera2 import Picamera2

In [ ]:
# 설정
# TARGET_IP = "100.75.235.29"     # 리마늘 IP 주소
TARGET_IP = "100.83.27.20"     # 내 IP 주소
TARGET_PORT = 5000

In [ ]:
picam2 = Picamera2()

# 카메라 설정 (1640x1232)
config = picam2.create_video_configuration(main={"size": (1640, 1232)})
picam2.configure(config)
picam2.start()

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)


In [ ]:
try:
    print(f"Sending images to {TARGET_IP}:{TARGET_PORT}")

    frame = picam2.capture_array()
    resized_frame = cv2.resize(frame, (640,512))

    frame_bgr = cv2.cvtColor(resized_frame, cv2.COLOR_RGB2BGR)
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 80]
    result, enimg = cv2.imencode('.jpg', frame_bgr, encode_param)

    if result:
        data = np.array(enimg)
        string_data = data.tobytes()
        sock.sendto(string_data, (TARGET_IP, TARGET_PORT))
        print(f"Image sent: {len(string_data)} bytes")
finally:
    picam2.stop()
    sock.close()

# 작동 확인 완료

# 이번엔 GStreamer을 이용해 영상을 보내보자

python에서 H.264로 rpicam2의 영상을 압축하고 RTSP로 특정 IP로 보내는 server 측의 코드를 생성해주삼

In [ ]:
import gi
gi.require_version('Gst', '1.0')
gi.require_version('GstRtspServer', '1.0')
from gi.repository import Gst, GstRtspServer, GLib

class MiraeRobotRTSPServer:
    def __init__(self):
        Gst.init(None)
        self.server = GstRtspServer.RTSPServer()
        self.server.set_service("8554")
        self.factory = GstRtspServer.RTSPMediaFactory()
        
        # [성공했던 파이프라인 완벽 이식]
        # 1. format=I420 명시 (명령어 성공의 핵심)
        # 2. x264enc 앞에 videoconvert를 중복 배치하여 안전성 확보
        # 3. pay0 설정에 config-interval 추가
        pipeline = (
            "( libcamerasrc ! video/x-raw, format=I420, width=1640, height=1232, framerate=30/1 ! "
            "videoconvert ! "
            "x264enc tune=zerolatency bitrate=4000 speed-preset=ultrafast ! "
            "rtph264pay name=pay0 pt=96 config-interval=1 )"
        )
        
        self.factory.set_launch(pipeline)
        self.factory.set_shared(True)
        self.server.get_mount_points().add_factory("/stream", self.factory)
        self.server.attach(None)
        
        print("RTSP Server is ready at rtsp://100.123.48.3:8554/stream")

if __name__ == '__main__':
    server = MiraeRobotRTSPServer()
    loop = GLib.MainLoop()
    try:
        loop.run()
    except KeyboardInterrupt:
        print("\nStopping Server...")

RTSP Server is ready at rtsp://100.123.48.3:8554/stream


[3:26:21.985841281] [13661]  INFO Camera camera_manager.cpp:340 libcamera v0.6.0+rpt20251202
[3:26:21.998247014] [13662]  INFO RPI pisp.cpp:720 libpisp version 1.3.0
[3:26:22.002062738] [13662]  INFO IPAProxy ipa_proxy.cpp:180 Using tuning file /usr/share/libcamera/ipa/rpi/pisp/imx219.json
[3:26:22.012036681] [13662]  INFO Camera camera_manager.cpp:223 Adding camera '/base/axi/pcie@1000120000/rp1/i2c@80000/imx219@10' for pipeline handler rpi/pisp
[3:26:22.012060384] [13662]  INFO RPI pisp.cpp:1181 Registered camera /base/axi/pcie@1000120000/rp1/i2c@80000/imx219@10 to CFE device /dev/media2 and ISP device /dev/media0 using PiSP variant BCM2712_C0
[3:26:22.016748132] [13665]  INFO Camera camera.cpp:1215 configuring streams: (0) 1640x1232-YUV420/Rec709
[3:26:22.016893966] [13662]  INFO RPI pisp.cpp:1485 Sensor: /base/axi/pcie@1000120000/rp1/i2c@80000/imx219@10 - Selected sensor format: 1640x1232-SBGGR10_1X10/RAW - Selected CFE format: 1640x1232-PC1B/RAW



Stopping Server...


## 윈도우에선...

`gst-launch-1.0 -v rtspsrc location=rtsp://100.123.48.3:8554/stream protocols=tcp retry=50 timeout=5000000 latency=500 ! rtph264depay ! avdec_h264 ! autovideosink`

- powershell에서 이렇게 하면 영상 뜬다잉
- 방화벽 켜도 됨